In [1]:
import torch 
import torch.nn as nn
from scipy import signal
import matplotlib.pyplot as plt
import pandas as pd
import neurokit2 as nk
import glob
import os
import wfdb
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import os
import torch
import pandas as pd
from time import sleep
from tqdm import tqdm
import os, tqdm
from sklearn.metrics import *
import json

import pickle

import keras
import matplotlib.pyplot as plt
import numpy as np
import os
from keras.callbacks import LearningRateScheduler
from keras.layers import Conv1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.models import Input, Model
from keras.regularizers import l2
from scipy.interpolate import splev, splrep
import pandas as pd

In [2]:
base_dir = "./data/SA-2.0.0/"

ir = 3 # interpolate interval
before = 2
after = 2

# normalize
scaler = lambda arr: (arr - np.min(arr)) / (np.max(arr) - np.min(arr))


def load_data():
    tm = np.arange(0, (before + 1 + after) * 60, step=1 / float(ir))

    with open(os.path.join(base_dir, "apnea-ecg.pkl"), 'rb') as f: # read preprocessing result
        apnea_ecg = pickle.load(f)

    x_train = []
    o_train, y_train = apnea_ecg["o_train"], apnea_ecg["y_train"]
    groups_train = apnea_ecg["groups_train"]
    for i in range(len(o_train)):
        (rri_tm, rri_signal), (ampl_tm, ampl_siganl) = o_train[i]
		# Curve interpolation
        rri_interp_signal = splev(tm, splrep(rri_tm, scaler(rri_signal), k=3), ext=1) 
        ampl_interp_signal = splev(tm, splrep(ampl_tm, scaler(ampl_siganl), k=3), ext=1)
        x_train.append([rri_interp_signal, ampl_interp_signal])
    x_train = np.array(x_train, dtype="float32").transpose((0, 2, 1)) # convert to numpy format
    y_train = np.array(y_train, dtype="float32")

    x_test = []
    o_test, y_test = apnea_ecg["o_test"], apnea_ecg["y_test"]
    groups_test = apnea_ecg["groups_test"]
    for i in range(len(o_test)):
        (rri_tm, rri_signal), (ampl_tm, ampl_siganl) = o_test[i]
		# Curve interpolation
        rri_interp_signal = splev(tm, splrep(rri_tm, scaler(rri_signal), k=3), ext=1)
        ampl_interp_signal = splev(tm, splrep(ampl_tm, scaler(ampl_siganl), k=3), ext=1)
        x_test.append([rri_interp_signal, ampl_interp_signal])
    x_test = np.array(x_test, dtype="float32").transpose((0, 2, 1))
    y_test = np.array(y_test, dtype="float32")

    return x_train, y_train, groups_train, x_test, y_test, groups_test

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [4]:
import modelzoo as zoo
model = zoo.se_resnext50().to(device)

In [5]:
x_train, y_train, groups_train, x_test, y_test, groups_test = load_data()
class Sleep_Agnea_DB(Dataset):
    def __init__(self, data, label):
        self.data=data
        self.label=label
        

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data= self.data[idx].transpose()
        label=int(self.label[idx])
        return data, label
batch_size=128
train_data=Sleep_Agnea_DB(x_train,y_train)
valid_data=Sleep_Agnea_DB(x_test,y_test)
train_loader=DataLoader(train_data,batch_size=batch_size)
valid_loader=DataLoader(valid_data,batch_size=batch_size)

In [6]:

epochs, epoch_stop_decay = 50, 20
ckp_path="/home/vishc2/tuna/ecg/ecg_ver2/data"

history = {
        "train": {"loss": [], "f1": []}, 
        "val": {"loss": [], "f1": []}, 
        "lr": []
    }

best_f1 = 0.0

In [7]:
lr=1e-4
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay= 5e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, eta_min=0.1*lr, T_max=30)

In [8]:
# CosineAnnealingWarmRestarts

In [9]:

for epoch in range(1, epochs + 1):
    print("epoch {:2}/{:2}".format(epoch, epochs) + "\n" + "-"*16)

    model.train()
    running_loss = 0.0
    running_labels, running_preds = [], []
    for ecgs, labels in tqdm.tqdm(train_loader):
        
        ecgs, labels = ecgs.to(device), labels.to(device)

        optimizer.zero_grad()

        logits = model(ecgs.float())
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()*ecgs.size(0)
        labels, preds = list(labels.data.cpu().numpy()), list(torch.max(logits, 1)[1].detach().cpu().numpy())
        running_labels.extend(labels), running_preds.extend(preds)
        
        # print(np.argmax(test_labels, axis=1))

    epoch_loss, epoch_f1 = running_loss/len(train_loader.dataset), f1_score(running_labels, running_preds, average="macro")
    
    
    history["train"]["loss"].append(epoch_loss), history["train"]["f1"].append(epoch_f1)
    print("{:<5} - loss: {:.4f} - f1: {:.4f}".format("train", epoch_loss, epoch_f1))

    with torch.no_grad():
        model.eval()
        running_loss = 0.0
        running_labels, running_preds = [], []
        for ecgs, labels in tqdm.tqdm(valid_loader):
            ecgs, labels = ecgs.to(device), labels.to(device)

            logits = model(ecgs.float())
            loss = criterion(logits, labels)

            running_loss += loss.item()*ecgs.size(0)
            labels, preds = list(labels.data.cpu().numpy()), list(torch.max(logits, 1)[1].detach().cpu().numpy())
            running_labels.extend(labels), running_preds.extend(preds)

    epoch_loss, epoch_f1 = running_loss/len(valid_loader.dataset), f1_score(running_labels, running_preds, average="macro")
    
    history["val"]["loss"].append(epoch_loss), history["val"]["f1"].append(epoch_f1)
    print("{:<5} - loss: {:.4f} - f1: {:.4f}".format("val", epoch_loss, epoch_f1))

    history["lr"].append(optimizer.param_groups[0]["lr"])
    if epoch <= epoch_stop_decay:
        scheduler.step()

    if epoch_f1 > best_f1:
        best_f1 = epoch_f1
        torch.save(model.state_dict(), "{}/best_1.pt".format(ckp_path))

with open("{}/history_best-1.json".format(ckp_path), "w") as f:
    json.dump(history, f)

print("\nEvaluation ...")
print("="*64)

model.load_state_dict(torch.load("{}/best_1.pt".format(ckp_path)))
with torch.no_grad():
    model.eval()
    running_labels, running_preds = [], []
    for ecgs, labels in tqdm.tqdm(valid_loader):
        ecgs, labels = ecgs.to(device), labels.to(device)

        logits = model(ecgs.float())

        labels, preds = list(labels.data.cpu().numpy()), list(torch.max(logits, 1)[1].detach().cpu().numpy())
        running_labels.extend(labels), running_preds.extend(preds)

print("confusion-matrix:")
print(confusion_matrix(
    running_labels, running_preds
))
print("classification-report:")
print(classification_report(
    running_labels, running_preds
    , digits=3
))

print("\nFinish !!!")

epoch  1/50
----------------


 49%|████▉     | 64/131 [00:07<00:07,  9.23it/s]